In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
%matplotlib tk
# import pyvista as pv
# from pyvista import examples

In [56]:
font = {'family' : 'Helvetica',
        'weight' : 'bold',
        'size'   : 14}

mpl.rc('font', **font)

In [15]:
def m3s_km3yr(q):
    '''convert discharge in m3/s to discahrge in km3/yr for bqart'''
    con_q = q*(86400*365.25)*1e-9
    return con_q

### Inputs

##### constants

In [2]:
w = 0.0006 ## conversion constant to get sed fluxes in MT/yr
B = 1 ## lithology factor
pho_s = 1.7e3 ##sediment density in kg/m3

variates

In [3]:
T = 15 ## temperatures in °C, held constant
relief = np.append([0.2], np.arange(0.5, 5.5, 0.5)) # mountain relief in km
drainage_area = np.logspace(2, 7, num = 11) ##drainage area in m

In [32]:
# runoff = np.arange(0.1, 1.1, 0.1)*10e-3 ## runoff in km/yr
discharge = m3s_km3yr(1000) #input discharge in m3/s

### Define actual BQART formula

In [42]:
def bqart(w, B, T, q, area_grid, relief_grid):
    '''calculate sediment supply given inputs'''

    rhos = 1.7e3 ##sediment density kgm3
    # area_grid, relief_grid = np.meshgrid(area, relief)

    if T >= 2:
        qsy = w*B*T* q**0.31 * area_grid**0.5 * relief_grid
    else: 
        qsy = 2*w*B* q**0.31 * area_grid**0.5 * relief_grid
    
    qs = qsy/rhos # 10^8kgm3/yr

    return qs

Set initial conditions

In [43]:
relief = np.append([0.2], np.arange(0.5, 5.5, 0.5)) # mountain relief in km
drainage_area = np.logspace(2, 7, num = 11) ##drainage area in m
Temp = 15

A, R = np.meshgrid(drainage_area, relief)
Qsy = w*B*T*(discharge)**0.31 * A**0.5 * R
Qs = Qsy/pho_s

In [84]:
fig, ax = plt.subplots(figsize = (10,10), dpi = 200)

## plot intial figure tobe manipulated

# fig, ax = plt.subplots()
cont = ax.contourf(A, R, Qs, cmap = 'gist_earth', levels = 15)
clines = ax.contour(A, R, Qs, colors = 'k', linewidths = 2, levels = 15)
ax.clabel(clines, colors = 'k')

# for c in clines.collections():
#         clines.remove()
        
fig.colorbar(cont, ax = ax)
ax.set_xlabel('Area, km2')
ax.set_ylabel('Relief, km')

# adjust the main plot to make room for the sliders
fig.subplots_adjust(bottom=0.3)

# Make a horizontal slider to control the frequency.
axtemp = fig.add_axes([0.25, 0.2, 0.65, 0.03])
temp_slider = Slider(
    ax=axtemp,
    label='Temperature, °C',
    valmin=-10,
    valmax=40,
    valinit=Temp,
)

# Make a vertically oriented slider to control the amplitude
axq = fig.add_axes([0.25, 0.15, 0.65, 0.03])
q_slider = Slider(
    ax=axq,
    label="Discharge",
    valmin=10,
    valmax=1e4,
    valinit=discharge,
    orientation="horizontal"
)


# The function to be called anytime a slider's value changes
def update(val):
    
    cont = ax.contourf(A, R, bqart(w, B, T = temp_slider.val, q = q_slider.val, area_grid = A, relief_grid = R),
                       cmap = 'gist_earth', levels = 15)
    
    
#     for c in clines.collections:
#         c.remove()  # removes only the contours, leaves the rest intact
    
#     clines = ax.contour(A, R, bqart(w, B, T = temp_slider.val, q = q_slider.val, area_grid = A, relief_grid = R),
#                         colors = 'k', linewidths = 2)
    
    
#     # labs = ax.clabel(clines, colors = 'k')
    # cont.set_ydata(bqart(w, B, freq_slider.val, amp_slider.val, area, relief))
    fig.canvas.draw_idle()
    # fig.canvas.draw()


# register the update function with each slider
temp_slider.on_changed(update)
q_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    temp_slider.reset()
    q_slider.reset()
button.on_clicked(reset)

plt.show()